In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from tqdm import tqdm
import string


In [30]:
csv_path = '../../data/telegram/scrape.telegram_forwardsGT30_TokensGT100.csv'
df_telegram = pd.read_csv(csv_path)
df_telegram_copy = df_telegram.copy()

df_head_copy = df_telegram_copy.head()
print(df_head_copy)

   Unnamed: 0                           chat      country state city  views  \
0           0  https://t.me/helpfulinfoforua  Switzerland   NaN  NaN   7159   
1           8    https://t.me/ukraine_reborn  Switzerland   NaN  NaN   2803   
2          10    https://t.me/ukraine_reborn  Switzerland   NaN  NaN   4659   
3          15    https://t.me/ukraine_reborn  Switzerland   NaN  NaN   2084   
4          20    https://t.me/ukraine_reborn  Switzerland   NaN  NaN   6558   

   forwards  replies                                        messageText  \
0        42      124  👋Вітаємо на каналі КОРИСНО (HELPFUL) Швейцарсь...   
1        54        0  🇺🇦 Пікнік. Український борщ (Женева)\n\nАсоціа...   
2        61       11  🇺🇦 Запис на літні заходи Passeport vacances \n...   
3        73        0  🇺🇦 Консультація з освіти у Швейцарії\n\nOlga B...   
4       123        1  🇺🇦 Особливості роботи в Швейцарії \n\nЗапрошує...   

  predicted_class messageDate  
0    Volunteering  2022-07-14  
1       Ed

In [3]:
nltk.download('stopwords')
nltk.download('punkt')
# stopwords, add ukrainian stopwords from .txt file
stopWords = stopwords.words('english') 
for word in stopwords.words('german'):
    stopWords.append(word)
for word in stopwords.words('french'):
    stopWords.append(word)
for word in stopwords.words('italian'):
    stopWords.append(word)
for word in stopwords.words('russian'):
    stopWords.append(word)
with open("../../data/stopwords/stopwords_ua.txt") as file: #add ukrainian stopwords loaded from .txt file
    ukrstopWords = [line.rstrip() for line in file]
for stopwords in ukrstopWords:
    stopWords.append(stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\86183\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\86183\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


data preprocessing

In [32]:
print(df_telegram_copy['messageText'][0])

# removing stopwords
def remove_stopwords(text):
    if len(text) > 50:
        new_text = []
        for word in text.split():
            if word in stopWords:
                continue
            else:
                new_text.append(word)
        return ' '.join(new_text)
    else:
        return text

# removing HTML Tags
def remove_html(text):
    if len(text) > 50:
        remove_ = re.compile('<.*?')
        return re.sub(remove_, r'', text)
    else:
        return text

# removing URLs
def remove_url(text):
    if len(text) > 50:
        re_url = re.compile('https?://\S+|www\.\S+')
        return re_url.sub('', text)
    else:
        return text

# removing emojis, from git
def remove_emojis(text):
    if len(text) > 50:
        emoji_pattern = re.compile(pattern="["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F700-\U0001F77F"  # alchemical symbols
            u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
            u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
            u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
            u"\U0001FA00-\U0001FA6F"  # Chess Symbols
            u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
            u"\U00002702-\U000027B0"  # Dingbats
            u"\U000024C2-\U0001F251"
            "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r'', text)
    else:
        return text

# lowercasing
def convert_lowercase(text):
    if len(text) > 50:
        text = text.lower()
        return text
    else:
        return text

# removing punctuation
def remove_punctuation(sentences):
    cleaned_sentences = []
    translator = str.maketrans('', '', string.punctuation)
    for sentence in sentences:
        cleaned_sentence = sentence.translate(translator)
        cleaned_sentences.append(cleaned_sentence)
    return cleaned_sentences

# tokenization 
def tokenize(text):
    if len(text) > 50:
        text = text.replace("\n", " ")
        text = sent_tokenize(text)
        return text
    else:
        return text

df_telegram_copy['messageText'] = df_telegram_copy['messageText'].apply(remove_stopwords).apply(remove_html).apply(remove_url).apply(remove_emojis).apply(convert_lowercase).apply(tokenize).apply(remove_punctuation)

print(df_telegram_copy['messageText'][0])

👋Вітаємо на каналі КОРИСНО (HELPFUL) Швейцарського Червоного Хреста.\nМи — група волонтерів, яка допоможе вам знайти відповіді на ваші запитання. Ми намагаємось відповідати впродовж 24 годин. Відповіді на запитання надаються з понеділка по п’ятницю. \n\nПерш ніж поставити своє запитання, просимо вас спочатку переглянути веб-сайт КОРИСНО (HELPFUL): helpful.redcross.ch\n\nНа цьому каналі можна використовувати українську, російську, англійську та державні мови Швейцарії. Ми відповідатимемо здебільшого державними мовами Швейцарії та пропонуватимемо перекласти наші відповіді. Для цього скористайтеся функцією «перекладу», яку пропонує Telegram. Указівки та коментарі наведено нижче.\n\n⚠️Оскільки це публічний канал, ваші коментарі бачитимуть всі бажаючі.\nЯкщо у вас є запитання, що передбачає розкриття особистої інформації, просимо вас додати в коментарі значок ✉️.\nТоді ми надішлемо вам приватне повідомлення в зашифрованому чаті.\n\n📍Відповіді на деякі питання були надані в приватних повідом

In [33]:
df_telegram_copy['messageText'][90]

['18 липня європейський парламент підтримав рішення розширення визнання українських посвідчень водія',
 'n nвідтепер єс можна n n користуватись українським посвідченням водія необхідності обміну періоду перебування захистом n n користуватися посвідченнями водія старого зразка оформленими кирилицею наявності завіреного перекладу міжнародного посвідчення водія',
 'але документ підтвердження особи n n  користуватись виданими вперше 2 українським посвідченням водія разі закінчення терміну дії n n отримати складання іспитів тимчасове на період перебування захистом посвідчення водія зразка єс разі втрати викраденняукраїнського посвідчення водія',
 'n nчерез 5 днів публікації відповідний регламент набуде чинностіn nперевіряти українські документи європейські колеги зможуть онлайн сайті сервісного центру мвс застосунок дія',
 'nдетальніше']

TextRank with TF-IDF

In [24]:
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer

class TfIdfTextSummarizer:
    def __init__(self):
        # Initialize with TF-IDF vectorizer
        self.tfidf_vectorizer = TfidfVectorizer()
        
    # Calculate TF-IDF similarity
    def calculate_similarity(self, sentence1, sentence2):
        if not sentence1.strip() or not sentence2.strip():
            return 0.0
        try:
            tfidf_matrix = self.tfidf_vectorizer.fit_transform([sentence1, sentence2])
            similarity_score = (tfidf_matrix * tfidf_matrix.T).toarray()[0, 1]
            return similarity_score
        except Exception as e:
            print("Error calculating similarity for the following sentences:")
            print("Sentence 1:", sentence1)
            print("Sentence 2:", sentence2)
            print("Error message:", str(e))
            return 0.0
        
    # Build TextRank graph
    def build_text_rank_graph(self, sentences):
        G = nx.Graph() 
        for sentence in sentences:
            G.add_node(sentence)
        for sentence1 in sentences:
            for sentence2 in sentences:
                if sentence1 != sentence2:
                    similarity_score = self.calculate_similarity(sentence1, sentence2)
                    G.add_edge(sentence1, sentence2, weight=similarity_score)
        return G

    # generate summarization with textrank
    def text_rank_summary(self, text):
        sentences = text
        graph = self.build_text_rank_graph(sentences)
        scores = nx.pagerank(graph, weight='weight')
        max_score = max(scores.values())
        top_sentences = [sentence for sentence, score in scores.items() if score == max_score]
        summary = top_sentences[0] 
        return summary, max_score

In [26]:
output_path = '../../data/summarization/prediction_textrank-sum.csv'
df_telegram_copy_2 = df_telegram.copy()
summarizer = TfIdfTextSummarizer()
df_telegram_copy_2['summary'], df_telegram_copy_2['max_score'] = zip(*df_telegram_copy['messageText'].apply(summarizer.text_rank_summary))
df_telegram_copy_2.to_csv(output_path, index=False, encoding='utf-8')

TextRank with Bert model

In [34]:

import networkx as nx
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import torch

class BertTextSummarizer:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
        self.model = AutoModel.from_pretrained("bert-base-multilingual-cased")
        
    # calculate sentence embeddings using bert
    def embedding_bert(self, sentence):
        input_ids = self.tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)["input_ids"]
        with torch.no_grad():
            outputs = self.model(input_ids)
            sentence_embedding = torch.mean(outputs.last_hidden_state, dim=1).squeeze(0).numpy()
        return sentence_embedding
    
    # Build TextRank graph using BERT embeddings
    def build_text_rank_graph(self, sentences):
        G = nx.Graph()
        num_sentences = len(sentences)
        sentence_embedding = [self.embedding_bert(sentence) for sentence in sentences]
        for i in range(num_sentences):
            for j in range(i + 1, num_sentences):
                embedding1 = sentence_embedding[i]
                embedding2 = sentence_embedding[j]
                similarity_score = cosine_similarity([embedding1], [embedding2])[0][0]
                G.add_edge(sentences[i], sentences[j], weight=similarity_score)
        return G
    
    # generate summarization with textrank
    def text_rank_summary(self, text):
        sentences = text
        graph = self.build_text_rank_graph(sentences)
        scores = nx.pagerank(graph, weight='weight')
        if not scores:
            return "", 0.0
        max_score = max(scores.values())
        top_sentences = [sentence for sentence, score in scores.items() if score == max_score]
        summary = top_sentences[0] 
        return summary, max_score

In [35]:
output_path = '../../data/summarization/prediction_textrank-sum-bert.csv'
df_telegram_copy_bert = df_telegram.copy()
summarizer_bert = BertTextSummarizer()
df_telegram_copy_bert['summary'], df_telegram_copy_bert['max_score'] = zip(*df_telegram_copy['messageText'].apply(summarizer_bert.text_rank_summary))
df_telegram_copy_bert.to_csv(output_path, index=False, encoding='utf-8')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
